# Data Cleaning

Master thesis of Nikolai Popov, MAE-2025


In [1]:
# Libraries import
import pandas as pd # dataframes
import glob # for reading several files one by one
import numpy as np # for matrices/vectors
from tqdm import tqdm # for progress bar
import gc # to delete a dataframe from the memory
import re # regular expression to work with strings
import ast # for correcting the list type
import warnings # to supress warninings
warnings.simplefilter(action='ignore', category=Warning)

In [2]:
# Mount Google Drive in Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define the path to the file in Google Drive
file_path = "/content/drive/My Drive/Coding/Merged_raw_data.csv"

# Load CSV with correct encoding and delimiter
raw_dataset = pd.read_csv(file_path, sep=",", on_bad_lines="skip", low_memory=False)

# Show first few rows
raw_dataset.head()

,SparkID,ShortNameEn,INN,okveds,OKVED,Address_Address,Address_Region,Address_City,Address_Longitude,Address_Latitude,...,Status_Code,Status_Date,Year,Staff,Source,Form_1_Field_1150,Form_2_Field_2120,Form_2_Field_2400,Form_4_Field_4121,Form_2_Field_2110
0,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2011,NaN,CUR,NaN,NaN,NaN,NaN,NaN
1,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2011,NaN,NXT,NaN,NaN,NaN,NaN,NaN
2,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2012,NaN,CUR,NaN,NaN,NaN,NaN,NaN
3,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2012,NaN,NXT,NaN,NaN,NaN,NaN,NaN
4,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2013,NaN,CUR,NaN,NaN,NaN,NaN,NaN


## Variables description

'Address_Address' - address of the enterprise headquarters; <br>
'Address_City' - city of the enterprise headquarters; <br>
'Address_Latitude' - latitude of the headquarters;  <br>
'Address_Longitude' - longitude of the headquarters; <br>
'Address_Region' - region of the headquarters;<br>
'Form_1_Field_1150' - fixed assets, RUB (form of the financial statement - proxy for capital); <br>
'Form_2_Field_2110' - revenue, RUB (proxy for output); <br>
'Form_2_Field_2120' - cost of sales, RUB;<br>
'Form_2_Field_2400' - net profit, RUB; <br>
'Form_4_Field_4121' - payments to suppliers and contractors for raw materials, goods, work, and services, RUB;<br>
'INN' - taxpayer number (Russian Federation); <br>
'OCVED' - main activity of a firm code;<br>
'OCVEDs' - all activities codes;<br>
'ShortNameEn' - short name of a company in English;<br>
'Source' - initial source of data; CUR means that the data for the year t was reported at the beginning of year t+1; NXT - corrected and reported at the beginning of year t+2; <br>
'SparkID' - SPARK's indentifier for a firm;<br>
'Staff' - number of employees (available only from 2018);<br>
'Status_Code' - detailed firm's status (see SPARK's documentation);<br>
'Status_Date' - date when the status was verified;<br>
'Status_IsActing' - dummy for the firm being active;<br>
'Year' - year of observation and financial statement report.<br>

## Cleaning

Test for the correctness of columns.

In [4]:
# Set of columns, manually exstrated from one of the CSV files
columns_set_original = {
    "SparkID", "ShortNameEn", "INN", "okveds", "OKVED",
    "Address_Address", "Address_Region", "Address_City",
    "Address_Longitude", "Address_Latitude", "Status_IsActing",
    "Status_Code", "Status_Date", "Year", "Staff", "Source",
    "Form_1_Field_1150", "Form_2_Field_2120", "Form_2_Field_2400", "Form_4_Field_4121", "Form_2_Field_2110"
}

# Set of columns, existing in the merged dataframe
columns_set_dataframe = set(raw_dataset.columns)

# print differences
differences = columns_set_original ^ columns_set_dataframe
print(f"Differences between sets: {differences}")

Differences between sets: set()


Well, no discrepancies.

In [5]:
raw_dataset.head(3)

,SparkID,ShortNameEn,INN,okveds,OKVED,Address_Address,Address_Region,Address_City,Address_Longitude,Address_Latitude,...,Status_Code,Status_Date,Year,Staff,Source,Form_1_Field_1150,Form_2_Field_2120,Form_2_Field_2400,Form_4_Field_4121,Form_2_Field_2110
0,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2011,NaN,CUR,NaN,NaN,NaN,NaN,NaN
1,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2011,NaN,NXT,NaN,NaN,NaN,NaN,NaN
2,325,SSZH,NaN,"['68.32', '68.32.2']",01.19,"г. Москва, проспект Олимпийский, д. 10 корп. 1...",г. Москва,г. Москва,"37,622423","55,777127",...,36,NaN,2012,NaN,CUR,NaN,NaN,NaN,NaN,NaN


In [6]:
print(f"Number of observations in the raw merged dataset: {raw_dataset.shape[0]}")

Number of observations in the raw merged dataset: 8131760


### Coordinates

In [7]:
# Change the separating sign for the latitude/longitude and convert to float
raw_dataset["Address_Longitude"] = raw_dataset["Address_Longitude"].astype(str).str.replace(",", ".").astype(float)
raw_dataset["Address_Latitude"] = raw_dataset["Address_Latitude"].astype(str).str.replace(",", ".").astype(float)

# Creat column of coordinates
raw_dataset["coord"] = raw_dataset["Address_Latitude"].astype(str) + ", " + raw_dataset["Address_Longitude"].astype(str)
raw_dataset = raw_dataset.drop(columns=\
                 ["Address_Longitude", "Address_Latitude"])

raw_dataset["coord"].sample(3)

,coord
3528660,"43.641474, 43.953189"
4017603,"53.217237, 50.648303"
63874,"55.814361, 47.162073"


### Data types

In [8]:
# See the types of the columns
raw_dataset.dtypes

,0
SparkID,int64
ShortNameEn,object
INN,float64
okveds,object
OKVED,object
Address_Address,object
Address_Region,object
Address_City,object
Status_IsActing,bool
Status_Code,int64


In [9]:
# Change columns to the proper data format
raw_dataset["Status_Date"] = pd.to_datetime(raw_dataset["Status_Date"], format="%Y-%m-%d")
raw_dataset["Staff"] = pd.to_numeric(raw_dataset["Staff"],
                                     errors="coerce").astype("Int64") # skips NaNs
raw_dataset["INN"] = pd.to_numeric(raw_dataset["INN"],
                                     errors="coerce").astype("Int64") # skips NaNs

### Renaming columns
Note that fixed assets = Capital <br>
revenue = Output

In [10]:
# Rename columns (according to financial statement)
raw_dataset = raw_dataset.rename(columns={
    "Form_1_Field_1150": "Capital",
    "Form_2_Field_2110": "Output",
    "Form_2_Field_2120": "Cost_of_sales",
    "Form_2_Field_2400": "Net_Profit",
    "Form_4_Field_4121": "Payments",
    "OKVED": "Main_OKVED"
})

### Removing duplicate obs

Apparently, there are INN-Year duplicate observations: same data but from different sources ('Source' - for the description see the Variables Description section).

As 'Source' = 'NXT' is considered a bit more accurate than CUR (as the data could be correcting during an additional year), let's exclude dublicate observations for which 'Source' = 'CUR'.

In [11]:
raw_dataset = raw_dataset[(raw_dataset["Source"] == "NXT")]

## Datasets Formation

### Whole cleaned dataset

In [12]:
raw_dataset.columns

Index(['SparkID', 'ShortNameEn', 'INN', 'okveds', 'Main_OKVED',
       'Address_Address', 'Address_Region', 'Address_City', 'Status_IsActing',
       'Status_Code', 'Status_Date', 'Year', 'Staff', 'Source', 'Capital',
       'Cost_of_sales', 'Net_Profit', 'Payments', 'Output', 'coord'],
      dtype='object')

In [13]:
print(f"The number of observations in the whole dataset: {raw_dataset.shape[0]}")

The number of observations in the whole dataset: 4065880


In [14]:
raw_dataset.to_csv('/content/drive/My Drive/Coding/whole_dataset_file_collab.csv', index=False, encoding="utf-8-sig")

### Balanced panel


Apparently, there are INN-Year duplicate observations: same data but from different sources ('Source').

#### Necessary columns
Exclude all unnecessary columns

In [15]:
balanced_df = raw_dataset[['INN', 'Year', 'okveds', 'coord', 'Capital', 'Output']]

# Delete the old dataset
del raw_dataset

# Run garbage collection to free up memory
gc.collect() # prints the number of deleted referene objects

0

#### Years range

In [16]:
# Find the range of years
balanced_df['Year'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021,
       2022, 2023])

From below the analysis is bounded by the data limitations, while from above it is bounded by the local validity of the DID - the further the period is from the treatment period, the less believable the assumption of parallel trends becomes (due to new shocks, structural breaks etc.). As my empirical model is a spatial DID with the treatment year of 2014, I take the years from 2011 to 2017: 3 years prior to 2014 and after. Actually, I might have taken 2018 into account as well (but not 2019 - COVID is a structural break which violates the parallel trends assumption for sure), but due to the requirement of my dataset being a balanced panel, I decided not to loose additional 10% of the firms.

In [17]:
# leave only required years
balanced_df = balanced_df[(balanced_df["Year"] >= 2011) &\
    (balanced_df["Year"] <= 2017)]

#### Balancing

Balanced panel is required for spatial DID models.

Due to the specification of the empirical model, Capital, Output should be strictly greater than zero, as
they are used in ln().

In [18]:
# Keep only rows where 'capital' > 0 and 'output' > 0
balanced_df = balanced_df[(balanced_df['Capital'] > 0) & (balanced_df['Output'] > 0)]

Print the number of missing values in each column

In [19]:
print(f'Dataframe shape: {balanced_df.shape}')
print(balanced_df.isna().sum())

Dataframe shape: (238767, 6)
INN        0
Year       0
okveds     0
coord      0
Capital    0
Output     0
dtype: int64


In [20]:
# Check if every firm has a permanent adress (required for spatial weights
# to be time-invariant)
num_adress = balanced_df.groupby("INN")["coord"].nunique()

# Find identifiers where multiple unique values exist
changed_adress_firms = num_adress[num_adress > 1]
changed_adress_firms

,coord
INN,
5029004536,2


In [21]:
# Filter out these firms from the DataFrame
balanced_df = balanced_df[~balanced_df["INN"].isin(changed_adress_firms)]

Now lets find the firms which don't have observations for each year of observation.

In [22]:
# Define the expected range of years
expected_years = set(range(balanced_df["Year"].min(),
                           balanced_df["Year"].max() + 1))

# Find firms missing one or more years
missing_years_firms = balanced_df.groupby("INN")["Year"].\
apply(lambda x: expected_years - set(x))

# Extract firms that are missing years
firms_to_exclude =\
set(missing_years_firms[missing_years_firms.apply(len) > 0].index)

# Exclude these firms from the dataset
balanced_df = balanced_df[~balanced_df["INN"].isin(firms_to_exclude)]
print(f"Previous number of firms: {len(missing_years_firms)}")
print(f"Number of firms excluded: {len(firms_to_exclude)}")
print(f"New number of firms: {len(missing_years_firms) -len(firms_to_exclude)}")
print(f"Number of years: {7}")
print(f"Expected number of obs. (balanced panel):\n \
# years * # firms: {(len(missing_years_firms) -len(firms_to_exclude))*7}")
print(f"Actual number of obs.: {balanced_df.shape[0]}")

Previous number of firms: 58964
Number of firms excluded: 49823
New number of firms: 9141
Number of years: 7
Expected number of obs. (balanced panel):
 # years * # firms: 63987
Actual number of obs.: 63987


#### Post dummy

In [23]:
# Create a new column "Post_d" which is 1 if Year >= 2014, else 0
balanced_df["Post_d"] = (balanced_df["Year"] >= 2014).astype(int)
balanced_df[["Year", "Post_d"]].head()

,Year,Post_d
27,2011,0
29,2012,0
31,2013,0
33,2014,1
35,2015,1


#### Treatment dummy

OCVED - firm's activity code; TNVED - good's code. They have to be matched manually. The world's standards, which correspinds: OCVED - Nice, TN_VED (6 digits) - HS. Note that there is not direct correspondance between TNVED and OCVED. There is also OKPD (European analog is CPA) - it's the code, linking the goods produced to company activity code. There is a direct correspondance with TNVED (I used this webiste to convert TNVED to OKPD: https://classinform.ru/perevod-tnved-v-okpd2/search.html). Even though there is still no direct correspondance between OKPD and OCVED, they coinside up to 4-th digit - meaning that they can be matched, even though only on the first levels of classification.


Steps to implement the mapping:
1) Convert OCVED to OKPD;
2) Convert OKPD to OCVED;

The procedure is done manually to increase the presition of the ambiguous matching. Note that due to the matching on only 4 digits, exclusions to the sanctioned categories cannot be taken into account; the whole category considered embargoed instead.

See the link to the official Russian Government statement about TN_VED codes (first 6 digits of which corresponds to the HS system) of emabargoed goods:

https://base.garant.ru/70712500/53f89421bbdaf741eb2d1ecc4ddb4c33/

In [24]:
# manually matched okveds corresponding to activities of fimrs, which were
# directly affected by embargo (counter-sanctions)

embargoed_sectors = {'01.46', '10.11', '10.12', '10.13',
'03.11', '03.12', '03.22', '10.20', '10.51', '01.13',
'01.11', '01.26', '01.28', '10.31', '10.39', '01.25',
'01.22', '01.23', '01.21', '01.24', '10.41', '08.93'}

In [25]:
balanced_df['okveds'].head()

,okveds
27,"['47.21', '47.22', '47.29.11', '49.4']"
29,"['47.21', '47.22', '47.29.11', '49.4']"
31,"['47.21', '47.22', '47.29.11', '49.4']"
33,"['47.21', '47.22', '47.29.11', '49.4']"
35,"['47.21', '47.22', '47.29.11', '49.4']"


Truncate all the okveds to 4 digits (for the proper matching).

In [26]:
# Ensure that the okveds column contains actual lists, not string representations.
def ensure_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception as e:
            print("Conversion error:", e)
            return None
    return x
balanced_df['okveds'] = balanced_df['okveds'].apply(ensure_list)

def modify_okved(codes_list):
    return [re.sub(r'^(\d+\.\d+)\.\d+$', r'\1', code) for code in codes_list]

# Apply the modification
balanced_df['okveds'] = balanced_df['okveds'].apply(modify_okved)

print(balanced_df['okveds'].head())

27    [47.21, 47.22, 47.29, 49.4]
29    [47.21, 47.22, 47.29, 49.4]
31    [47.21, 47.22, 47.29, 49.4]
33    [47.21, 47.22, 47.29, 49.4]
35    [47.21, 47.22, 47.29, 49.4]
Name: okveds, dtype: object


See whether firms have the same okveds over the years:

In [27]:
# Convert the list of okveds into a canonical form.
# Sort the codes and convert them to a tuple so that order doesn't affect equality.
def canonical_okved(okved_list):
    if okved_list is None:
        return None
    return tuple(sorted(okved_list))

balanced_df['okveds_canonical'] = balanced_df['okveds'].apply(canonical_okved)

# Group by INN and check if the set of okveds remains the same over the years.
def is_consistent(group):
    # If there's only one unique canonical set across the group's rows, it's consistent.
    return group['okveds_canonical'].nunique() == 1

# Create a DataFrame with consistency check results
consistency = balanced_df.groupby('INN').apply(is_consistent).reset_index()
consistency.columns = ['INN', 'consistent']

# Count the number of inconsistent firms
num_inconsistent = (consistency['consistent'] == False).sum()

balanced_df = balanced_df.drop(columns=\
                 ["okveds_canonical"])
print("Number of inconsistent firms:", num_inconsistent)

Number of inconsistent firms: 0


With that out of the way, let's mark treated (embargoed) firms. Note that if at least one of the okveds of the firms coincides with an element form the embargoed list => this firm is treated as embargoed (it's a simplification).

In [28]:
balanced_df['Treated_d'] = balanced_df['okveds'].apply(
    lambda codes: int(bool(set(codes).intersection(embargoed_sectors)))
)
# Reset index
balanced_df = balanced_df.reset_index(drop=True)

# Display a few rows to verify the results
print(balanced_df[['INN', 'Year', 'okveds', 'Treated_d']].head())

         INN  Year                       okveds  Treated_d
0  245008247  2011  [47.21, 47.22, 47.29, 49.4]          0
1  245008247  2012  [47.21, 47.22, 47.29, 49.4]          0
2  245008247  2013  [47.21, 47.22, 47.29, 49.4]          0
3  245008247  2014  [47.21, 47.22, 47.29, 49.4]          0
4  245008247  2015  [47.21, 47.22, 47.29, 49.4]          0


In [29]:
print(f"Total number of observations: {balanced_df.shape[0]}")
print(f"Number of treated observations: {balanced_df['Treated_d'].sum()}")

Total number of observations: 63987
Number of treated observations: 47950


In [30]:
balanced_df.sample(3)

,INN,Year,okveds,coord,Capital,Output,Post_d,Treated_d
27882,7130010740,2012,"[01.11, 01.13, 01.19, 10.12, 10.12, 10.13, 10....","54.167553, 37.445677",1.479240e+09,1.769463e+09,0,1
53286,3128033189,2013,"[01.11, 01.11, 10.61, 10.71, 10.82, 10.91, 11....","51.337915, 37.841843",2.568240e+08,4.107607e+09,0,1
54601,5072724594,2012,"[10.1, 10.11, 10.11, 10.11, 10.51, 10.51, 10.5...","54.965557, 39.027719",1.054100e+08,2.935800e+07,0,1


In [31]:
balanced_df.to_csv('/content/drive/My Drive/Coding/balanced_dataset_file_collab.csv', index=False, encoding="utf-8-sig")


### The list of INN

In [32]:
# Get unique firm IDs (INN)
unique_firms = balanced_df["INN"].unique()

# Convert to a standard NumPy array (optional, but without it didn't work)
unique_firms = unique_firms.astype(np.int64)  # Ensures consistent integer format

# Print results
print(f"Number of unique firms: {len(unique_firms)}")

Number of unique firms: 9141


In [33]:
INN_list_file = "C:/Users/Popov/Documents/Research/Volchkova_thesis/Data/Cleaned_datasets/INN_list_data_test.csv"

# Convert unique firms list to a DataFrame
unique_firms_df = pd.DataFrame(unique_firms, columns=["INN"])

unique_firms_df.to_csv('/content/drive/My Drive/Coding/unique_firms_df_collab.csv', index=False, encoding="utf-8-sig")
